# Merchant EDA – Univariate & Multivariate
Goal: Analyze merchant-level attributes and their relationship with historical fraud


In [2]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mastercard_challenge.visualization.visualize import dist_visualisation, plot_categorical_distributions

In [3]:
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

DATA_PROCESSED_DIR = Path("../data/processed")
DATA_INTERIM_DIR = Path("../data/interim")

In [4]:
df = pd.read_parquet(DATA_PROCESSED_DIR / "merchants.parquet")

In [10]:
print("Initial shape:", df.shape)

Initial shape: (1000, 8)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   merchant_id                     1000 non-null   object 
 1   category                        1000 non-null   object 
 2   country                         1000 non-null   object 
 3   trust_score                     1000 non-null   float64
 4   number_of_alerts_last_6_months  1000 non-null   int64  
 5   avg_transaction_amount          1000 non-null   float64
 6   account_age_months              1000 non-null   int64  
 7   has_fraud_history               1000 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [6]:
df.head()

,merchant_id,category,country,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history
0,M0001,travel,Austria,1.000000,3,97.23,84,0
1,M0002,clothing,Poland,0.689695,2,142.71,93,1
2,M0003,electronics,Czech Republic,0.834610,5,67.83,65,0
3,M0004,electronics,Belgium,0.607229,1,16.77,9,1
4,M0005,grocery,Ireland,0.641901,2,38.56,100,0


In [9]:
df.describe()

,trust_score,number_of_alerts_last_6_months,avg_transaction_amount,account_age_months,has_fraud_history
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,0.682568,3.06100,81.883090,61.526000,0.52000
std,0.194092,1.72286,79.187529,33.453617,0.49985
min,0.000000,0.00000,0.120000,6.000000,0.00000
25%,0.545615,2.00000,24.772500,32.000000,0.00000
50%,0.697414,3.00000,58.205000,61.000000,1.00000
75%,0.825599,4.00000,117.192500,92.000000,1.00000
max,1.000000,9.00000,709.040000,119.000000,1.00000
